In [ ]:
!pip install -U xformers torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install diffusers["torch"] transformers peft
!pip install matplotlib rich Pillow
!pip install Cython

In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install localtunnel

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
console = Console()

In [ ]:
repo_list = ['John6666/prefect-pony-xl-v1-sdxl' , 'yodayo-ai/clandestine-xl-1.0']

In [8]:
root_path   = "/content/drive/MyDrive/StableDiffusion/"
output_path = "/content/drive/MyDrive/StableDiffusion/output"
lora_path   = "/content/drive/MyDrive/StableDiffusion/lora"
repo_url    = repo_list[1]

In [9]:
%load_ext sql

In [10]:
%sql sqlite:////content/drive/MyDrive/StableDiffusion/database/session.db

In [ ]:
%sql CREATE TABLE IF NOT EXISTS SESSION(ID INTEGER PRIMARY KEY AUTOINCREMENT, PROMPT TEXT NOT NULL, NEGATIVE_PROMPT TEXT NOT NULL);

In [12]:
def configure_gpu():
    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = True

In [13]:
def configure_pipeline(device='cuda', clip_skip=2):
    dtype = torch.float16 if device == 'cuda' else torch.float32
    pipe = StableDiffusionXLPipeline.from_pretrained(
        repo_url,
        torch_dtype=dtype,
        use_safetensors=True,
        custom_pipeline="lpw_stable_diffusion_xl",
        add_watermarker=False,
    )
    pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

    if device == 'cuda':
        configure_gpu()
        pipe.enable_xformers_memory_efficient_attention()
    else:
        raise ValueError("Unsupported device type. Please use 'cuda'.")

    # Configure clip skip if needed
    if hasattr(pipe, 'config'):
        pipe.config.clip_skip = clip_skip
    else:
        print("The pipeline does not support clip_skip configuration.")

    pipe.to(device)

    return pipe

In [ ]:
pipe = configure_pipeline()
pipe.run_safety_checker = False

In [23]:
import safetensors.torch

def reshape_lora_weights(lora_weights, expected_shape):
    reshaped_weights = {}
    for name, weight in lora_weights.items():
        if weight.shape != expected_shape and len(weight.shape) > 2:
            reshaped_weights[name] = weight.view(expected_shape)
        else:
            reshaped_weights[name] = weight
    return reshaped_weights

def load_lora_weights_with_reshape(model, lora_path):
    lora_weights = safetensors.torch.load_file(lora_path)
    expected_shape = torch.Size([32, 640])
    lora_weights = reshape_lora_weights(lora_weights, expected_shape)
    for name, param in model.named_parameters():
        if name in lora_weights:
            param.data += lora_weights[name].to(param.device)

load_lora_weights_with_reshape(pipe.unet, os.path.join(lora_path, 'intricate_details_Pony.safetensors'))
load_lora_weights_with_reshape(pipe.unet, os.path.join(lora_path, 'NEGATIVE_HANDS.safetensors'))

In [ ]:
result = %sql SELECT PROMPT , NEGATIVE_PROMPT FROM SESSION;

In [ ]:
result = [item for item in result]
print(result)

In [33]:
last_prompt = result[-1][0] if result else ''
last_negative_prompt = result[-1][1]if result else ''

In [34]:
main_prompt_widget = widgets.Textarea(value=last_prompt, placeholder='Positive Prompt', layout=widgets.Layout(width='55%', height='200px'))
negative_prompt_widget = widgets.Textarea(value=last_negative_prompt, placeholder='Negative Prompt', layout=widgets.Layout(width='55%', height='200px'))
guidance_scale_widget = widgets.FloatSlider(value=6.0, min=1.0, max=20.0, step=0.1, description='Guidance Scale:', layout=widgets.Layout(width='55%'))
inference_steps_widget = widgets.IntSlider(value=30, min=1, max=100, step=1, description='Inference Steps:', layout=widgets.Layout(width='55%'))
width_widget = widgets.IntText(value=784, description='Width:', layout=widgets.Layout(width='55%'))
height_widget = widgets.IntText(value=1352, description='Height:', layout=widgets.Layout(width='55%'))
submit = widgets.Button(description='Start', button_style='primary')

In [36]:
def generate_random_string(length=12):
    letters = string.ascii_uppercase + string.digits
    return ''.join(random.choice(letters) for _ in range(length))

def on_submit(event):
    plt.close('all')
    console.print("[yellow] Prepering Action [/]")
    images = []

    for _ in range(4):
        random_string = generate_random_string()
        date_string = datetime.now().strftime("%Y%m%d")
        filename = f"{random_string}-{date_string}.png"
        filename_path = os.path.join(root_path , 'output' , filename)

        image = pipe(main_prompt_widget.value,negative_prompt=negative_prompt_widget.value,
            width=width_widget.value,height=height_widget.value,guidance_scale=guidance_scale_widget.value,
            num_inference_steps=inference_steps_widget.value
        ).images[0]

        image.save(filename_path)
        images.append(image)

        insert_query = f"""
          INSERT INTO SESSION (PROMPT, NEGATIVE_PROMPT)
          VALUES ('{main_prompt_widget.value}', '{negative_prompt_widget.value}')
        """

        %sql {insert_query}


    fig, axs = plt.subplots(2, 2, figsize=(10, 10))
    for i, ax in enumerate(axs.flat):
        ax.imshow(images[i])
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [20]:
from threading import Thread
from pyngrok import ngrok
import os

In [21]:
ngrok.set_auth_token("2c32JTKQCKzLIj9smvoA1iivalj_3uksU7WFHz7PMFuDbctbX")

In [22]:
def run():
    os.system("streamlit run /content/drive/MyDrive/StableDiffusion/web.py --server.port 8501")

In [23]:
thread = Thread(target=run)
thread.start()

In [25]:
public_url = ngrok.connect(addr='8501' , proto='http' , bind_tls=True)
print(f'Running at {public_url}')

Running at NgrokTunnel: "https://8b57-34-106-248-66.ngrok-free.app" -> "http://localhost:8501"
